In [1]:
import math
import cv2
import numpy as np

In [2]:
dip = cv2.VideoCapture('assignment1.mp4', 0)

w = round(dip.get(cv2.CAP_PROP_FRAME_WIDTH))
h = round(dip.get(cv2.CAP_PROP_FRAME_HEIGHT)) 
fps = dip.get(cv2.CAP_PROP_FPS)        #크기와 초당 프레임 선언

print("width = %d, height = %d, fps = %d" %(w, h, fps))

width = 720, height = 404, fps = 29


In [3]:
ratio_w = float(input('가로 비율 : '))
ratio_h = float(input('세로 비율 : '))
degree = float(input('회전 각도(-180 ~ 180) : '))
print("역방향 = 1, 순방향 = 2")
n = int(input('변환 방법 : '))

가로 비율 :  1
세로 비율 :  1
회전 각도(-180 ~ 180) :  45


역방향 = 1, 순방향 = 2


변환 방법 :  1


In [4]:
# 파이썬 내 연산을 위해 radian으로 변환, 영상 특성에 따라 음수로 변환
radian = -math.radians(degree) 
a = math.cos(-radian)
b = math.sin(-radian)
#scaled는 가로/세로비를 적용한 새 크기
scaled_w = w*ratio_w 
scaled_h = h*ratio_h 
if n == 1 :
    if degree <= 90 and degree >= 0 : 
        d1 = 0
        d2 = scaled_w*math.sin(-radian)
    elif degree > 90 and degree <= 180 :
        d1 = scaled_w*math.cos(math.radians(180-degree))
        d2 = scaled_w*math.sin(math.radians(180-degree))+scaled_h*math.cos(math.radians(180-degree))
    elif degree < 0 and degree >= -90 :    
        d1 = scaled_h*math.sin(radian)
        d2 = 0
    elif degree < -90 and degree >= -180 :
        radian = math.radians(180+degree)
    else :
        print("지정된 범위 내의 값을 입력해 주세요.")

In [5]:
#new_w/h는 회전과 확대/축소를 모두 적용한 크기
new_w = math.ceil(abs(scaled_w*math.cos(radian))+abs(scaled_h*math.sin(radian))) 
new_h = math.ceil(abs(scaled_w*math.sin(radian))+abs(scaled_h*math.cos(radian))) 

if degree < -90 and degree >= -180 and n == 1 :
    d1 = new_w
    d2 = scaled_h*math.cos(radian)

fourcc = cv2.VideoWriter_fourcc(*'DIVX') 
out = cv2.VideoWriter('D:\\assignment1\\output.avi', fourcc, 30.0, (new_w,new_h))

In [6]:
if n == 1 :
    rotation_matrix = np.array([[a, b, (1-a)*(d1)-b*(d2)],
                                [-b, a, b*(d1)+(1-a)*(d2)]])
    transf_matrix = np.array([[1, 0, d1],[0, 1, d2]])
else :
    rotation_matrix = np.array([[a, b, (1-a)*(new_w/2)-b*(new_h/2)],
                                [-b, a, b*(new_w/2)+(1-a)*(new_h/2)]])
    transf_matrix = np.array([[1, 0, (new_w-scaled_w)/2],[0, 1, (new_h-scaled_h)/2]])

In [7]:
def forward(img) :
    new_img = np.full((new_h, new_w, 3), (0,0,0), dtype=np.uint8)
    for i in range(0, h):
        y = i*ratio_h     
        for j in range(0, w):
            x = j*ratio_w
            new_x = round(math.cos(radian)*x-math.sin(radian)*y
                          -math.cos(radian)*(scaled_w/2)+math.sin(radian)*(scaled_h/2) + new_w/2)
            new_y = round(math.sin(radian)*x+math.cos(radian)*y
                          -math.sin(radian)*(scaled_w/2)-math.cos(radian)*(scaled_h/2) + new_h/2)
            
            if new_y < new_h and new_y >= 0 and new_x < new_w and new_x >= 0 :
                new_img[new_y, new_x,:] = img[i][j][:] 
    return new_img        

In [8]:
def backward(img) :
    rsz_img = cv2.resize(img, dsize=(0,0), fx = ratio_w, fy = ratio_h)
    tsn_img = cv2.warpAffine(rsz_img, transf_matrix, (3*new_w, 3*new_h))
    new_img = cv2.warpAffine(tsn_img, rotation_matrix, (new_w, new_h))
    return new_img

In [9]:
while(dip.isOpened()) :
    ret, frame = dip.read()
    
    if ret :      
        if n == 1 :
            result = backward(frame)            
        elif n == 2 :
            result = forward(frame)
        cv2.imshow('nice', result)
        out.write(result)
                
        if cv2.waitKey(10) & 0xFF == ord('q') :
            break
    else :
        break
        
dip.release()
cv2.destroyAllWindows()